In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys

import numpy as np
from astropy.cosmology import Planck18
import py21cmfast as p21c

WDIR = os.environ['DM21CM_DIR']
sys.path.append(WDIR)
from dm21cm.dm_params import DMParams
from dm21cm.evolve import evolve

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colormaps as cms
mpl.rc_file(f"{WDIR}/matplotlibrc")

/home/zixuan/anaconda3/envs/dm21cm-ann/lib/python3.9/site-packages/py21cmfast/_cfg.py:58: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/home/zixuan/anaconda3/envs/dm21cm-ann/lib/python3.9/site-packages/py21cmfast/_cfg.py:42: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")


In [2]:
# check that the environment variables are set correctly
print('DM21CM_DIR:     ', os.environ['DM21CM_DIR'])      # Set to the working directory of DM21cm. This directory should contain DM21cm's `README.md`
print('DM21CM_DATA_DIR:', os.environ['DM21CM_DATA_DIR']) # Set to directory containing DM21cm's data files. This directory should contain `abscissas.h5`
print('DH_DIR:         ', os.environ['DH_DIR'])          # Set to the working directory of DarkHistory. This directory should contain DarkHistory's `README.md`
print('DH_DATA_DIR:    ', os.environ['DH_DATA_DIR'])     # Set to directory containing DarkHistory's data files. This directory should contain `binning.h5`
print('P21C_CACHE_DIR: ', os.environ['P21C_CACHE_DIR'])  # Set to directory where you want to store the 21cmFAST cache. The files for a particular run will be stored in a subdirectory of this directory.

DM21CM_DIR:      /home/zixuan/DM21cm-ann/
DM21CM_DATA_DIR: /home/zixuan/DM21cm-data/
DH_DIR:          /home/zixuan/DarkHistory-DM21cm-ann
DH_DATA_DIR:     /home/zixuan/DHdata_v1_1_full/
P21C_CACHE_DIR:  /home/zixuan/DM21cm-ann-cache/


## 1. Evolve `DM21cm`

We first specify parameters for `21cmFAST`, including simulation configuration and all astrophysical parameters.

In [3]:
p21c_initial_conditions = p21c.initial_conditions(
    user_params = p21c.UserParams(
        HII_DIM = 64,
        BOX_LEN = 256, # [conformal Mpc]
        N_THREADS = 8,
    ),
    cosmo_params = p21c.CosmoParams(
        OMm = Planck18.Om0,
        OMb = Planck18.Ob0,
        POWER_INDEX = Planck18.meta['n'],
        SIGMA_8 = Planck18.meta['sigma8'],
        hlittle = Planck18.h,
    ),
    random_seed = 12345,
    write = True,
)

p21c_astro_params = p21c.AstroParams(
    F_STAR10 = -1.25,
    F_STAR7_MINI = -2.5,
    ALPHA_STAR = 0.5,
    ALPHA_STAR_MINI = 0.0,
    t_STAR = 0.5,
    F_ESC10 = -1.35,
    F_ESC7_MINI = -1.35,
    ALPHA_ESC = -0.3,
    L_X = 40.5,
    L_X_MINI = 40.5,
    NU_X_THRESH = 500,
    A_LW = 2.0,
)

# optionally set global parameters
p21c.global_params.CLUMPING_FACTOR = 1.

/home/zixuan/anaconda3/envs/dm21cm-ann/lib/python3.9/site-packages/py21cmfast/inputs.py:487: UserWarning: The USE_INTERPOLATION_TABLES setting has changed in v3.1.2 to be default True. You can likely ignore this warning, but if you relied onhaving USE_INTERPOLATION_TABLES=False by *default*, please set it explicitly. To silence this warning, set it explicitly to True. Thiswarning will be removed in v4.
  warnings.warn(


Now we can start the `DM21cm` run. We recommend starting from a high redshift such as $z=45$. \
The default time step of `DM21cm` is $\Delta z/(1+z)=0.002$. \
We recommend a coarser time step for `21cmFAST` $\Delta z/(1+z)=1.002^{10}-1\approx0.02$. \
This can be achieved by setting `subcycling_factor=10`, which is the default value. \
Set `dm_params`, and we are ready to go.

In [4]:
return_dict_base = evolve(
    run_name = 'base',
    z_start = 45.,
    z_end = 5.,
    subcycle_factor = 10,
    dm_params = DMParams(
        mode='decay',
        primary='phot_delta',
        m_DM=1e6, # [eV]
        lifetime=1e100, # [s]
    ),
    p21c_initial_conditions = p21c_initial_conditions,
    p21c_astro_params = p21c_astro_params,
    no_injection=True,
)

[2024-07-05 01:18:28,219] INFO: dm21cm.evolve: Using 21cmFAST version 0.1.dev1586+g6f60f17
[2024-07-05 01:18:28,220] INFO: dm21cm.evolve: Cache dir: /home/zixuan/DM21cm-ann-cache//base
[2024-07-05 01:18:32,829] INFO: dm21cm.dh_wrapper: Running DarkHistory to generate initial conditions...
[2024-07-05 01:18:32,830] INFO: darkhistory.config: Using data at /home/zixuan/DHdata_v1_1_full/
[2024-07-05 01:19:25,866] INFO: darkhistory.config: Loaded deposition transfer functions.
[2024-07-05 01:19:28,367] INFO: darkhistory.config: Loaded ICS transfer functions.
100%|█████████████████████████████████████████| 431/431 [00:32<00:00, 13.15it/s]
[2024-07-05 01:20:01,527] INFO: dm21cm.dh_wrapper: Saved DarkHistory initial conditions.
100%|███████████████████████████████████████| 1010/1010 [08:00<00:00,  2.10it/s]
/home/zixuan/anaconda3/envs/dm21cm-ann/lib/python3.9/site-packages/py21cmfast/_utils.py:821: UserWarning: Trying to remove array that isn't yet created: hires_vx
  warnings.warn(f"Trying to

21cmFAST    : 4.7760 +/- 1.4841 s * 100 steps : 477.5959 s
lightcone   : 3.9217 s * 1 steps : 3.9217 s


In [5]:
return_dict_decay1 = evolve(
    run_name = 'decay1',
    z_start = 45.,
    z_end = 5.,
    subcycle_factor = 10,
    dm_params = DMParams(
        mode='decay',
        primary='phot_delta',
        m_DM=5.46e5, # [eV]
        lifetime=4.09e25, # [s]
    ),
    p21c_initial_conditions = p21c_initial_conditions,
    p21c_astro_params = p21c_astro_params,
)

[2024-07-05 01:28:05,481] INFO: dm21cm.evolve: Using 21cmFAST version 0.1.dev1586+g6f60f17
[2024-07-05 01:28:05,482] INFO: dm21cm.evolve: Cache dir: /home/zixuan/DM21cm-ann-cache//decay1
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
[2024-07-05 01:28:10,279] INFO: dm21cm.tf_wrapper: Loaded photon transfer functions.
[2024-07-05 01:28:10,279] INFO: dm21cm.tf_wrapper: Skipping electron transfer functions.
[2024-07-05 01:28:13,100] INFO: dm21cm.dh_wrapper: Running DarkHistory to generate initial conditions...
[2024-07-05 01:28:13,101] INFO: darkhistory.config: Using data at /home/zixuan/DHdata_v1_1_full/
[2024-07-05 01:28:25,095] INFO: darkhistory.config: Loaded deposition transfer functions.
[2024-07-05 01:28:27,622] INFO: darkhistory.config: Loaded ICS transfer functions.
100%|█████████████████████████████████████████| 431/431 [00:34<00:00, 12.64it/s]
[2024-07-05 01:29:02,173] INFO: dm21cm.dh_wrapper: Saved DarkHistory initial conditions.
1

xray        : 7.2602 +/- 2.1968 s * 1009 steps : 7325.5094 s
bath+dm     : 0.8183 +/- 0.0237 s * 1009 steps : 825.6638 s
prep next   : 0.0110 +/- 0.0011 s * 1009 steps : 11.1185 s
21cmFAST    : 4.7811 +/- 1.4813 s * 100 steps : 478.1133 s
lightcone   : 4.2138 s * 1 steps : 4.2138 s


In [7]:
return_dict_decay3 = evolve(
    run_name = 'decay3',
    z_start = 45.,
    z_end = 5.,
    subcycle_factor = 10,
    dm_params = DMParams(
        mode='decay',
        primary='phot_delta',
        m_DM=1.95e6, # [eV]
        lifetime=2.54e23, # [s]
    ),
    p21c_initial_conditions = p21c_initial_conditions,
    p21c_astro_params = p21c_astro_params,
)

[2024-07-05 06:19:20,471] INFO: dm21cm.evolve: Using 21cmFAST version 0.1.dev1586+g6f60f17
[2024-07-05 06:19:20,472] INFO: dm21cm.evolve: Cache dir: /home/zixuan/DM21cm-ann-cache//decay3
[2024-07-05 06:19:27,823] INFO: dm21cm.tf_wrapper: Loaded photon transfer functions.
[2024-07-05 06:19:27,824] INFO: dm21cm.tf_wrapper: Skipping electron transfer functions.
[2024-07-05 06:19:30,276] INFO: dm21cm.dh_wrapper: Running DarkHistory to generate initial conditions...
[2024-07-05 06:19:30,276] INFO: darkhistory.config: Using data at /home/zixuan/DHdata_v1_1_full/
[2024-07-05 06:19:46,083] INFO: darkhistory.config: Loaded deposition transfer functions.
[2024-07-05 06:19:49,099] INFO: darkhistory.config: Loaded ICS transfer functions.
100%|█████████████████████████████████████████| 431/431 [00:32<00:00, 13.41it/s]
[2024-07-05 06:20:21,688] INFO: dm21cm.dh_wrapper: Saved DarkHistory initial conditions.
100%|█████████████████████████████████████| 1010/1010 [2:26:58<00:00,  8.73s/it]


xray        : 7.4254 +/- 2.3692 s * 1009 steps : 7492.2550 s
bath+dm     : 0.8242 +/- 0.0309 s * 1009 steps : 831.6430 s
prep next   : 0.0110 +/- 0.0011 s * 1009 steps : 11.1131 s
21cmFAST    : 4.8037 +/- 1.4933 s * 100 steps : 480.3730 s
lightcone   : 3.9982 s * 1 steps : 3.9982 s


In [9]:
return_dict_decay5 = evolve(
    run_name = 'decay5',
    z_start = 45.,
    z_end = 5.,
    subcycle_factor = 10,
    dm_params = DMParams(
        mode='decay',
        primary='phot_delta',
        m_DM=2.63e6, # [eV]
        lifetime=7.59e22, # [s]
    ),
    p21c_initial_conditions = p21c_initial_conditions,
    p21c_astro_params = p21c_astro_params,
)

[2024-07-05 11:16:31,082] INFO: dm21cm.evolve: Using 21cmFAST version 0.1.dev1586+g6f60f17
[2024-07-05 11:16:31,083] INFO: dm21cm.evolve: Cache dir: /home/zixuan/DM21cm-ann-cache//decay5
[2024-07-05 11:16:35,395] INFO: dm21cm.tf_wrapper: Loaded photon transfer functions.
[2024-07-05 11:16:35,396] INFO: dm21cm.tf_wrapper: Skipping electron transfer functions.
[2024-07-05 11:16:37,863] INFO: dm21cm.dh_wrapper: Running DarkHistory to generate initial conditions...
[2024-07-05 11:16:37,863] INFO: darkhistory.config: Using data at /home/zixuan/DHdata_v1_1_full/
[2024-07-05 11:16:49,167] INFO: darkhistory.config: Loaded deposition transfer functions.
[2024-07-05 11:16:51,049] INFO: darkhistory.config: Loaded ICS transfer functions.
100%|█████████████████████████████████████████| 431/431 [00:32<00:00, 13.39it/s]
[2024-07-05 11:17:23,700] INFO: dm21cm.dh_wrapper: Saved DarkHistory initial conditions.
100%|█████████████████████████████████████| 1010/1010 [2:28:03<00:00,  8.80s/it]


xray        : 7.4744 +/- 2.3842 s * 1009 steps : 7541.6344 s
bath+dm     : 0.8361 +/- 0.0292 s * 1009 steps : 843.6532 s
prep next   : 0.0111 +/- 0.0011 s * 1009 steps : 11.2042 s
21cmFAST    : 4.8427 +/- 1.4862 s * 100 steps : 484.2659 s
lightcone   : 3.9776 s * 1 steps : 3.9776 s


you should see outputs that looks like this:
```
[YYYY-MM-DD HH:MM:SS,XXX] INFO: dm21cm.evolve: Using 21cmFAST version X.X.X
[YYYY-MM-DD HH:MM:SS,XXX] INFO: dm21cm.evolve: Cache dir: /path/to/cache_dir/test
[YYYY-MM-DD HH:MM:SS,XXX] INFO: dm21cm.tf_wrapper: Loaded photon transfer functions.
[YYYY-MM-DD HH:MM:SS,XXX] INFO: dm21cm.tf_wrapper: Skipping electron transfer functions.
[YYYY-MM-DD HH:MM:SS,XXX] INFO: dm21cm.dh_wrapper: Running DarkHistory to generate initial conditions...
[YYYY-MM-DD HH:MM:SS,XXX] INFO: darkhistory.config: Using data at /path/to/DHdata_v1_1
[YYYY-MM-DD HH:MM:SS,XXX] INFO: darkhistory.config: Loaded deposition transfer functions.
[YYYY-MM-DD HH:MM:SS,XXX] INFO: darkhistory.config: Loaded ICS transfer functions.
100%|██████████| 431/431 [00:45<00:00,  9.50it/s]
[YYYY-MM-DD HH:MM:SS,XXX] INFO: dm21cm.dh_wrapper: Saved DarkHistory initial conditions.
100%|██████████| 1010/1010 [23:49<00:00,  1.42s/it]
xray        : 0.6174 +/- 0.1840 s * 1009 steps : 622.9507 s
bath+dm     : 0.0337 +/- 0.0007 s * 1009 steps : 34.0274 s
prep next   : 0.0189 +/- 0.0027 s * 1009 steps : 19.0329 s
21cmFAST    : 7.4479 +/- 2.2558 s * 100 steps : 744.7943 s
lightcone   : 6.9483 s * 1 steps : 6.9483 s
```

In the `cache_dir` directory, you will find the following files from `21cmFAST`: \
`InitialConditions_[hash]_r[seed].h5` \
`IonizedBox_[hash]_r[seed].h5` \
`TsBox_[hash]_r[seed].h5` \
`BrightnessTemp_[hash]_r[seed].h5` \
`PerturbedField_[hash]_r[seed].h5`

Aside from these, there will be files generated by `DM21cm`:

In [10]:
[f for f in os.listdir(f"{os.environ['P21C_CACHE_DIR']}/base") if f[:6] not in ['Initia', 'Ionize', 'TsBox_', 'Pertur', 'Bright']]

['global_records.npy', 'lightcones.h5', 'dh_init_soln.p']

`global_records.npy` : contains the same information as `return_dict['global']`. \
`lightcones.h5` : contains the same information as `return_dict['lightcone']`. \
`xray_box_cache.h5`: contains the cached xray relative luminosities due to DM injection. \
`xray_cache_snapshot.p`: contains a snapshot of the `XrayCacher` object (that controls `xray_box_cache.h5`). Used for resuming runs. \
`dh_init_soln.p`: contains `DarkHistory` solutions up to $\lesssim45$ for initialization of `DM21cm`.